Last Name:

First Name:

NOMA:


This completed assignment must be uploaded to **Moodle** before XX/XX/2020, in two formats:
1. the present `.ipynb` file, with your results, and
2. a "frozen" `.html` version, via `File>Download as>HTML (.html)`

Before starting this assignment, remember to **complete your Name and NOMA** in the first cell of this notebook

# Introduction

## Contents
This Notebook is your interface to use the Equilibrium Model, for which the code is found in the attached `.py` files. It is structured in 4 parts:
1. The description of the model and its underlying equations
2. JANAF data: some explanations on the thermochemical data used by the model, and observations on the reaction enthalpy and equilibrium constants
3. "How-to": the practical explanation on how to make use of the model and a small example
4. **Assignment**: this is the part where you will make your own analyses using the model, and summarize your conclusions and observations using Markdown cells.

## Required packages

To run the model, first make sure the following Python packages are installed on your computer:
- `numpy`
- `scipy`
- `pandas`
- `plotly`
- `molmass`
- `warnings`
- `tqdm`

**Run the next cell to import the model and packages**

In [ ]:
from functions_janaf import *
from functions_model import model_results, ER_optimization
from functions_parametric_analyses import parametric_analysis, parametric_analysis_2D, extract_1D_results, parametric_analysis_display, parametric_analysis_2D_display
from functions_plot import plot_var, plot_contour, plot_sankey, px, go

# MODEL DESCRIPTION

The equilibrium model that you will use to simulate biomass gasification is based on the combination of the global stoichiometric reaction, and intermediate reactions which are assumed to reach equilibrium, all given below.

$
\text{CH$_y$O$_x$N$_z$} + m \mathrm{H_2O}_l + s \mathrm{H_2O}_g + w\left(\mathrm{O_2} + \xi\mathrm{N_2}\right)\qquad \qquad \qquad \\
\qquad \qquad \qquad \Rightarrow a_1\mathrm{CO} + a_2\mathrm{CO_2} + a_4\mathrm{CH_4} + b_1\mathrm{H_2} + b_2\mathrm{H_2O}_g + \left(\frac{z}{2}+\xi w\right)\mathrm{N_2} + a_0\mathrm{C}_s
$

**Boudouard ($\mathrm{CO_2}$ gasification)**: $$\mathrm{C\mathit{_s} + CO_2 <=>2CO}$$
**Water gasification**: $$\mathrm{C\mathit{_s} + H_2O <=> CO + H_2}$$
**Shift (WGS - Water-Gas Shift)**: $$\mathrm{CO + H_2O <=> CO_2 + H_2}$$
**SMR (Steam Methane Reforming)**: $$\mathrm{CH_4 + H_2O <=> CO + 3H_2}$$
**Methanation**: $$\mathrm{C\mathit{_s} + 2H_2 <=> CH_4}$$

## Products composition

Assuming a given value of the equilibrium temperature $T_{eq}$ (= temperature of the products), these chemical reactions can be used to determine the **composition of the syngas** by solving *mass conservation* and *chemical equilibrium* equations. As there are *6 unknowns* for syngas composition ($a_1$, $a_2$, $a_4$, $b_1$, $b_2$ and $a_0$), *6 equations* are required: the conservation of C, H and O, and **three equilibrium equations**. Any choice of three *independent* reactions should yield identical results: here the chosen combination was WGS, SMR and Boudouard. The last equation is added to ensure the gas concentrations sum up to 1.

In situations where no solid Carbon is found in the products, the three reactions involving $\mathrm{C}_s$ should not be used. Simultaneously, an equation can be removed as $n_{\mathrm{C}_s}$ is no more unknown: Boudouard reaction equilibrium is hence not considered in that case.

**As a personal exercise, re-develop the system of equations below on your own**

**Model equations - syngas composition**:

\begin{align}
&x_{\mathrm{CO}} + x_{\mathrm{CO2}} + x_{\mathrm{CH_4}} - (1 - n_{\mathrm{C}_s})\frac{x_{N_2}}{n_{N_2}} = 0\\
&4 x_{\mathrm{CH_4}} + 2 x_{\mathrm{H_2}} + 2 x_{\mathrm{H_2O}_g}  - (y + 2m + 2s)\frac{x_{N_2}}{n_{N_2}} = 0\\
&x_{\mathrm{CO}} + 2 x_{\mathrm{CO_2}} + x_{\mathrm{H_2O}_g} - (x + m + s + 2w)\frac{x_{N_2}}{n_{N_2}} = 0\\
&x_{\mathrm{CO_2}}x_{\mathrm{H_2}} - x_{\mathrm{CO}}n_{\mathrm{H_2O}_g}K_{\text{Shift}}  = 0\\
&x_{\mathrm{CO}}x_{\mathrm{H_2}}^3 \left(\frac{1}{n_{gas}}\frac{p}{p^°}\right)^2 - x_{\mathrm{CH_4}}x_{\mathrm{H2O}_g}K_{\text{SMR}} = 0\\
&x_{\mathrm{CO}}^2 \frac{p}{p^°} - x_{\mathrm{CO_2}}K_{\text{Boudouard}} = 0 \qquad \qquad \text{(only if solid Carbon present in the Products)} \qquad  \qquad \\
&x_{\mathrm{CO}} + x_{\mathrm{CO_2}} + x_{\mathrm{CH_4}} + x_{\mathrm{H_2}} + x_{\mathrm{H_2O}_g} + x_{N_2} - 1 = 0\\
\end{align}

The unknowns are $x_{\mathrm{CO}}$, $x_{\mathrm{CO_2}}$, $x_{\mathrm{CH_4}}$, $x_{\mathrm{H_2}}$, $x_{\mathrm{H_2O}_g} $, $x_{N_2}$ and $n_{\mathrm{C}_s}$, where $x_i=\frac{n_i}{n_{gas}}$ with $n_{gas}=\frac{n_{N_2}}{x_{N_2}}$ the total number of gas moles, and $n_{N_2}=\frac{z}{2}+\xi w$


## Equilibrium temperature

The above system of equations is combined with **energy conservation**, used to determine $T_{eq}$ assuming an adiabatic reactor. This consists in equating the products enthalpy to the reactants enthalpy (LHV + sensible enthalpy). In an iterative process, energy conservation and the above model equations are solved until the equilibrium temperature and products composition is found.

The functions constituting the equilibrium model, described below, can be found in the file `functions_model.py`.

**Have a look at these functions to make sure you understand how the model works before starting the analyses below**.

## ER optimization

For each set of gasification parameters, there exists an optimum value of the equivalence ratio (ER = $\lambda$) that corresponds to the situation where all carbon is converted (no solid carbon remains), in which case the **cold gas efficiency (CGE)** is maximized. The CGE corresponds to the energy content of the dry and cold syngas, relative to the energy content in the biomass feedstock, as:

$$
\mathrm{CGE} = \dfrac{n_{\mathrm{CO}} \mathrm{LHV}_{\mathrm{CO}} + n_{\mathrm{H_2}} \mathrm{LHV}_{\mathrm{H_2}} + n_{\mathrm{CH_4}} \mathrm{LHV}_{\mathrm{CH_4}}}{\mathrm{LHV}_{\text{CH$_y$O$_x$N$_z$}}}
$$

If ER optimization is activated, the above model will be iterated over the parameter **ER** until the maximum **CGE** is obtained

# JANAF DATA

Thermochemical data for the model are taken from **NIST-JANAF** database, available at: https://janaf.nist.gov/

JANAF tables contain the following properties (amongst others) for a large number of molecules, at various temperatures, considering Standard state pressure p° = 1 bar and reference temperature Tr = 298.15 K:
- *Enthalpy*: `H-H(Tr)` [kJ/mol]
- *Formation enthalpy*: `delta-f H` [kJ/mol]
- *Equilibrium constant of formation*: `log Kf` [-] 

For this model, the required tables are provided in the folder JANAF and called automatically by the functions in `functions_janaf.py`.

**Have a look at these functions to make sure you understand the underlying data used by the model and how they are used.**

## Reaction enthalpy and equilibrium constants

*The below data might be useful to analyse the model results:*

In [ ]:
delta_r_H_disp = delta_r_H.to_frame(r"$\Delta_r H [\mathrm{kJ/mol}]$")\
                          .set_index(delta_r_H.index)

display(delta_r_H_disp)

In [ ]:
T = kelvin(np.arange(300,1100,20))
logKr = pd.DataFrame(index=T, columns=reactions.keys()).rename_axis('T')
for reaction in reactions:
    logKr[reaction] = log_Kr(reaction, T)
logKr.index = celsius(logKr.index)
logKr.rename_axis('Teq [°C]', axis=0, inplace=True)

px.line(logKr.reset_index().melt(id_vars='Teq [°C]',var_name='reaction',value_name='log Kr'),x='Teq [°C]',y='log Kr',color='reaction',
        title='Equilibrium constants of main gasification reactions')

# HOW TO USE THE MODEL

## Model execution

To perform simulations with the equilibrium model described above, the following **functions** are available. These functions are found in the files `functions_model.py` and `functions_parametric_analyses.py`.

- `model_results`: equilibrium model results for a given set of parameters
- `ER_optimization`: equilibrium model results for a given set of parameters, with ER optimization
- `parametric_analysis`: equilibrium model results with one varying parameter, with or without ER optimization
- `parametric_analysis_2D`: equilibrium model results with two varying parameters, with or without ER optimization

These functions accept the following **inputs**:
- `input_param` or `other_param`: dict of constant non-default parameters
- `var_ID`, `var1`, `var2`: name(s) of varying parameters for parametric analyses
- `var_values`, `vals1`, `vals2`: values (list, array or series) of the varying parameters for parametric analyses
- `ER_optim`: boolean - use ER optimization or not (for parametric analyses) [default=False]
- `Teq_min`: constrained minimum on the equilibrium temperature in the case of ER optimization, in Celsius [default=0]

These functions return **two outputs**:
- `param`: summary of the simulation parameters (only the non-varying parameters).
- `results`:
    - `Teq` = the equilibrium temperature
    - `dfReac` = reactants composition, temperature, enthalpy, LHV... (called `dfReacs` for parametric analyses)
    - `dfProd` = products composition, temperature, enthalpy, LHV... (called `dfProds` for parametric analyses)
    - `dfReac_tot` = reactants aggregated results (total / average as applicable)
    - `dfProd_dnc` = products aggregated results, excluding solid Carbon and H2O for dry gas basis (total / average)
    - `ER` = optimum equivalence ratio (only for parametric analyses with ER optimization)
    
For the detailed definition of all columns of these DataFrames, have a look at their assignment in the functions `initiate_constants`, `solve_temperature`, `calculate_energy_figures` and `aggregate_results`

According to the number of **varying parameters** (0, 1 or 2) and the number of **dimensions** of each result (0 for `Teq`, 2 for `dfReac` and `dfProd` and 1 for `dfReac_tot` and `dfProd_dnc`), the corresponding output will be either of a float, a Series, a DataFrame, a dict of DataFrames or a dict of dict of DataFrames.

To facilitate the extraction of results in the latter two cases, the following functions are available:
- `extract_1D_results`: extract results along one dimension of 2D results, producing results of a 1D analysis
- `extract_single_function_1D`: extract a column of dfProd or dfReac from 1D results, in the form of a DataFrame
- `extract_single_function_2D`: extract a column of dfReac, dfProd, dfReac_tot or dfProd_dnc from 2D results, in the form of a DataFrame

**Make sure to have a look at these functions to understand how to use them.**

## Input parameters

When calling either of the 4 model functions above, the following parameters can be either:
- **left as default** using the default values listed below (if not specified)
- **set to a constant value** by defining them in the `input_param` or `other_param` dict
- **made to vary** through a parametric analysis

After a simulation, you should verify the `param` output to make sure of the parameters used for the results.

**Biomass parameters:**

To specify non-default biomass parameters, the best way is to include in the `input_param` dict an element `biomass` defined using the function `biomass_data` (from `functions_janaf.py`). To specify the LHV, you must either give `LHV_kg` [kJ/kg] or `LHV_mol` [kJ/mol].

*Biomass parameters are not meant to vary in a parametric analysis.*

| Parameter | Default value | Unit |
|:--|--:|:--|
| x    | 0.66  ||
| y    | 1.44  ||
| z    | 0  ||
| LHV  | 18500  | kJ/kg |


**Model parameters and default values:**

These parameters can be set directly in the `input_param` or `other_param` dict.
They can be made to vary in parametric analyses.

| Parameter | Default value | Unit | Description |
|:--|--:|:--|:--|
| ER     | 0.30  | - | Equivalence Ratio ($\lambda$)|
| O2_air | 0.21  | - | molar content of O2 in air |
| M      | 0.1 | kg_H2O / kg_bm_dry | Biomass moisture content (dry basis) |
| SBR    | 0    | kg_H2O / kg_bm_dry | Steam to biomass ratio |
| p      | 1     | bara | Gasification pressure |
| Tair   | 25    | °C | Air inlet temperature |
| Tst   | 150    | °C | Steam inlet temperature |

## Results display

To display the results, the following functions are provided.

- `plot_var`: plot one or several columns of a Series or DataFrame against its index, with predefined layout
- `plot_contour`: creates a contour plot of the data of a DataFrame
- `plot_sankey`: creates a Sankey diagram of the conversion from dfReac to dfProd, for energy, C, H or O
- `parametric_analysis_display`: plot results of 1D parametric analysis
- `parametric_analysis_2D_display`: plot results of 2D parametric analysis

*Note*: Teq is converted to Celsius while CGE and other ratios are converted in % for display

## Example: Carbon gasification (no ER optimization)

In [ ]:
biomass = biomass_data(x=0,y=0,z=0,LHV_mol=LHV_species['C']) #LHV_species was defined in functions_janaf.py
input_param = dict(M=0, # dry carbon
                   biomass=biomass)

results, param = model_results(input_param)
display(param)
plot_sankey(results['dfReac'], results['dfProd'], 'energy')

In [ ]:
biomass = biomass_data(x=0,y=0,z=0,LHV_mol=LHV_species['C']) #LHV_species was defined in functions_janaf.py
input_param = dict(M=0, # dry carbon
                   biomass=biomass)

results, param = parametric_analysis('ER',[x/10 for x in range(1,10)],input_param)
display(param)
parametric_analysis_display(results, ['nmol','CGE'])

# ASSIGNMENT

## Introduction

The objective of this exploratory project is for you to play around with the equilibrium model: read the code to understand its underlying structure, try out different configurations, observe and analyse the results while keeping in mind the underlying assumptions, think about how to produce the best syngas, or how to valorize lower quality biomass, compare situations...

To provide your comments, observations, remarks, questions, conclusions, please use Markdown cells within the notebook, in addition to the graphs that you'll plot. **What we are interested in are your observations more than the graphs or figures that you'll obtain**. To analyse your results, you might amongst others want to use the graph of logKr, which will help you understand what shifts the syngas composition in one direction or another.

The questions are left relatively open on purpose: we expect you to use your own curiosity to dig around and raise interesting results or findings. The main goal is for you to get very familiar with the thermochemistry of biomass thermochemical gasification. We have written a few questions below to guide you, and we expect that you will make use of the last "free" section to try out other analyses.

**This assignment is not graded, but might be subject to questions during the examination.**

## Biomass gasification: impact of moisture content

### Biomass moisture level

For the following analyses, you will consider the gasification of woody biomass of (default) parameters $\mathrm{CH_{1.44}O_{0.66}}$ and LHV = 18.5 MJ/kg.

We want to evaluate the impact of **drying** on the quality of the gasification. Based on the model results, using the available functions, assess and compare the performance of gasification between:
- Fresh biomass - moisture content = 50%
- Air-dried biomass - moisture content = 20 to 40%
- Oven-dried biomass - moisture content = 5 to 10%
- Dry biomass (theoretical) -> moisture content = 0%

How does the gasification efficency CGE and syngas energy content LHV evolve with the moisture content of biomass? You can consider that the gasifier is operated at the optimal air-fuel ratio. 

**Observations**:

### Energetic cost of drying

Compare the gain of using dried biomass to the theoretical cost of drying it (considering only the required energy to evaporate the moisture, assuming all heat is transferred directly to the water).

What solutions can be used to dry biomass at the lowest possible energy cost?

**Observations**:



## Biomass: impact of steam injection

### Constant ER

Instead of moisture, it could be interesting to inject steam to the gasification process. Analyse the effect of steam injection quantity (`SBR`) and temperature (`Tst`) on the syngas characteristics, in particular on its LHV and composition, first with a constant air-fuel ratio.

**Observations**:


### Optimized ER

Compare the above results with the situation with optimized ER, closer to how a practical gasifier would be operated, trying to find the optimal air-fuel ratio. Analyse how the added steam now impacts the LHV and CGE, and relate it to the evolution of optimal ER.

In addition, observe the effect of high amounts of steam (SBR up to 1): what can you say about the reliability of the results, considering the products temperature Teq and **ER** the underlying assumptions of the equilibrium model?

**Observations**:


### Minimum Teq

In an actual gasifier, the syngas outlet temperature would normally never go below 600°C. This can be constrained in the equilibrium model, by setting the value of `Teq_min` (input argument, in °C). Try it out and observe the results. What would you conclude regarding the potential gains from steam injection on biomass gasification (syngas properties, efficiency...)?

**Observations**:


### Net energy gain

Based on your previous observations, calculate the net energy impact of injecting steam, considering the energy required to produce that steam. Assume a perfect boiler with no heat losses and feedwater at 25°C.

What would you conclude on the benefit of steam injection on gasification, if steam is produced using a natural gas steam boiler? What other options do you think would be feasible to inject steam yielding a net energy gain?

**Observations**:


## Additional analyses

For the last part of this assignment, choose any other variable(s) on which to perform an additional analysis.

You could, for example, analyse the impact of varying air preheating (`Tair`), oxygen enrichment (`O2_air`) or biomass parameters (`x`, `y`, `z`, `LHV`) or combinations of variables on the results of the model such as `LHV`, `CGE`, syngas composition (`nmol`, `x_drygas`), equilibrium temperature `Teq`, etc. 

### ...

**Observations**
